In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Import required libraries

In [ ]:
# import the necessary packages
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.utils import np_utils
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt

### Load Dataset

In [ ]:
train=pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test=pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [ ]:
display(train.shape)
display(test.shape)

### Show the image

In [ ]:
from PIL import Image
# create figure
fig = plt.figure(figsize=(15, 15))

# setting values to rows and column variables
rows = 3
columns = 3

for i in range(9):
    # Adds a subplot at the 1st position
    fig.add_subplot(rows, columns, i+1)
    img_arr=train.iloc[i,1:].values.reshape(28,28)
    plt.imshow(img_arr)
    plt.axis('off')
    plt.title("Label"+' '+str(train.iloc[i,0]))

### Reshape all the image sizes

In [ ]:
trainX = train.drop("label", axis = 1)
trainY = train["label"]

trainX = trainX.astype("float32") / 255.0
trainX = trainX.values.reshape(-1,28,28,1)

In [ ]:
testX = test.astype("float32") / 255.0
testX = testX.values.reshape(-1,28,28,1)

In [ ]:
display(testX.shape)
display(trainX.shape)
display(trainY.shape)

### Encoding target labels

In [ ]:
trainY.unique()## 10 classes

In [ ]:
#Encoding the output class label (One-Hot Encoding)
trainY=np_utils.to_categorical(trainY,10)

In [ ]:
trainY

### Create the Graph Model

In [ ]:
# Note you only need to define the input data shape with the first layer. 
# The last layers is a dense layer with softmax activation that classifies the 10 categories.


model = tf.keras.Sequential()
# Must define the input shape in the first layer of the neural network
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', input_shape=(28,28,1))) 
model.add(tf.keras.layers.MaxPooling2D(pool_size=2 , padding='same'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2 , padding='valid'))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(10, activation='softmax'))
# Take a look at the model summary
model.summary()

### Compile the model

In [ ]:
model.compile(loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])


# model.compile(loss='categorical_crossentropy',
#              optimizer='rmsprop',
#              metrics=['accuracy'])

### Fit the model

In [ ]:
model.fit(trainX, trainY ,batch_size=64,epochs=10,verbose=1)

In [ ]:
model.fit(trainX, trainY ,batch_size=64,epochs=20,verbose=1,initial_epoch=10)

### Predict the model on test set

In [ ]:
### predict 
test_pred=model.predict(testX)

In [ ]:
test_pred = np.argmax(test_pred,1)

In [ ]:
test_pred.shape

In [ ]:
test.shape

In [ ]:
test['label']=test_pred

In [ ]:
### File for submission
submission=pd.read_csv('/kaggle/input/digit-recognizer/sample_submission.csv')

In [ ]:
submission['Label']=test_pred

In [ ]:
submission

In [ ]:
submission.to_csv('DR_submission.csv',index=False)